In [ ]:
import os

directory = '/kaggle/input/learntools/'
for filename in os.listdir(directory):
    print(os.path.join(directory, filename))

# 1. MLP

**Tiền xử lý dữ liệu**

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

# Directories for train, valid, test data
train_data_dir = '/kaggle/input/learntools/train/'
validation_data_dir = '/kaggle/input/learntools/valid/'
test_data_dir = '/kaggle/input/learntools/test/'

# Read data from train, valid, and test sets
train_filenames, train_labels = read_data(os.path.join(train_data_dir, "_annotations.csv"))
test_filenames, test_labels = read_data(os.path.join(test_data_dir, "_annotations.csv"))
validation_filenames, validation_labels = read_data(os.path.join(validation_data_dir, "_annotations.csv"))


# Xử lý dữ liệu hình ảnh từ tập train
train_images = []
train_labels_list = []
for filename, label in zip(train_filenames, train_labels):
    image = cv2.imread(os.path.join(train_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    train_images.append(image)
    train_labels_list.append(label)

train_images = np.array(train_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
train_labels = pd.get_dummies(train_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập valid
valid_images = []
valid_labels_list = []
for filename, label in zip(validation_filenames, validation_labels):
    image = cv2.imread(os.path.join(validation_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    valid_images.append(image)
    valid_labels_list.append(label)

valid_images = np.array(valid_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
valid_labels = pd.get_dummies(valid_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
test_labels = pd.get_dummies(test_labels_list)  # Chuyển đổi nhãn sang one-hot encoding


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

def create_mlp_model():
    model = Sequential()

    # Flatten input layer
    model.add(Flatten(input_shape=(224, 224, 3)))  # Đầu vào có kích thước (224, 224, 3)

    # Fully connected layers
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))  # Dropout để tránh overfitting
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(5, activation='softmax'))

    return model

# Tạo mô hình MLP
model_mlp = create_mlp_model()

# Hiển thị summary của mô hình để kiểm tra số lượng tham số
model_mlp.summary()


In [ ]:
# Biên dịch mô hình
model_mlp.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history_mlp = model_mlp.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(valid_images, valid_labels))

# Lưu mô hình
model_mlp.save('/kaggle/working/classification_MLP_model.h5')

In [ ]:
import matplotlib.pyplot as plt

# Biểu đồ loss và accuracy
plt.plot(history_mlp.history['loss'], label='Training Loss')
plt.plot(history_mlp.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history_mlp.history['accuracy'], label='Training Accuracy')
plt.plot(history_mlp.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Đánh giá mô hình trên tập test
test_loss_mlp, test_accuracy_mlp = model_mlp.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_mlp)
print("Test Accuracy:", test_accuracy_mlp)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Xây dựng mô hình MLP cải tiến
def create_improved_mlp_model(input_shape, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=input_shape),
        Dropout(0.5),  # Dropout để tránh overfitting
        Dense(256, activation='relu'),
        Dropout(0.3),  
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Tạo mô hình
model_improved_mlp = create_improved_mlp_model(input_shape=(224*224*3,), num_classes=5)

# Biên dịch mô hình
model_improved_mlp.compile(optimizer=Adam(learning_rate=0.001),
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
# Huấn luyện mô hình
history_improved_mlp = model_improved_mlp.fit(train_images.reshape(-1, 224*224*3), 
                                              train_labels, 
                                              epochs=100, 
                                              batch_size=32, 
                                              validation_data=(valid_images.reshape(-1, 224*224*3), valid_labels))
# Biểu đồ loss và accuracy
plt.plot(history_improved_mlp.history['loss'], label='Training Loss')
plt.plot(history_improved_mlp.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history_improved_mlp.history['accuracy'], label='Training Accuracy')
plt.plot(history_improved_mlp.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Đánh giá mô hình trên tập test
test_loss_improved_mlp, test_accuracy_improved_mlp = model_improved_mlp.evaluate(test_images.reshape(-1, 224*224*3), test_labels)
print("Test Loss:", test_loss_improved_mlp)
print("Test Accuracy:", test_accuracy_improved_mlp)

In [ ]:
# Lưu mô hình
model_improved_mlp.save('/kaggle/working/classification_MLP_model_v2.h5')

# 2. CNN

**Tiền xử lý dữ liêu**

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

# Directories for train, valid, test data
train_data_dir = '/kaggle/input/learntools/train/'
validation_data_dir = '/kaggle/input/learntools/valid/'
test_data_dir = '/kaggle/input/learntools/test/'

# Read data from train, valid, and test sets
train_filenames, train_labels = read_data(os.path.join(train_data_dir, "_annotations.csv"))
test_filenames, test_labels = read_data(os.path.join(test_data_dir, "_annotations.csv"))
validation_filenames, validation_labels = read_data(os.path.join(validation_data_dir, "_annotations.csv"))


# Xử lý dữ liệu hình ảnh từ tập train
train_images = []
train_labels_list = []
for filename, label in zip(train_filenames, train_labels):
    image = cv2.imread(os.path.join(train_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    train_images.append(image)
    train_labels_list.append(label)

train_images = np.array(train_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
train_labels = pd.get_dummies(train_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập valid
valid_images = []
valid_labels_list = []
for filename, label in zip(validation_filenames, validation_labels):
    image = cv2.imread(os.path.join(validation_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    valid_images.append(image)
    valid_labels_list.append(label)

valid_images = np.array(valid_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
valid_labels = pd.get_dummies(valid_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
test_labels = pd.get_dummies(test_labels_list)  # Chuyển đổi nhãn sang one-hot encoding


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_simple_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Tạo mô hình
model_simple_cnn = create_simple_cnn_model(input_shape=(224, 224, 3), num_classes=5)

# Biên dịch mô hình
model_simple_cnn.compile(optimizer='adam',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# In tổng quan của mô hình
model_simple_cnn.summary()

In [ ]:
# Huấn luyện mô hình
history_simple_cnn = model_simple_cnn.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(valid_images, valid_labels))

# Biểu đồ loss và accuracy
import matplotlib.pyplot as plt

plt.plot(history_simple_cnn.history['loss'], label='Training Loss')
plt.plot(history_simple_cnn.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history_simple_cnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_simple_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Đánh giá mô hình trên tập test
test_loss_simple_cnn, test_accuracy_simple_cnn = model_simple_cnn.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_simple_cnn)
print("Test Accuracy:", test_accuracy_simple_cnn)


In [ ]:
# Lưu mô hình
model_simple_cnn.save('/kaggle/working/classification_CNN_model.h5')

In [ ]:
def create_improved_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Tạo mô hình cải thiện
model_improved_cnn = create_improved_cnn_model(input_shape=(224, 224, 3), num_classes=5)

# Biên dịch mô hình
model_improved_cnn.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])

# Huấn luyện mô hình
history_improved_cnn = model_improved_cnn.fit(train_images, train_labels, epochs=100, batch_size=32, validation_data=(valid_images, valid_labels))

# Đánh giá mô hình trên tập test
test_loss_improved_cnn, test_accuracy_improved_cnn = model_improved_cnn.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_improved_cnn)
print("Test Accuracy:", test_accuracy_improved_cnn)


In [ ]:
# Biểu đồ loss và accuracy
import matplotlib.pyplot as plt

plt.plot(history_improved_cnn.history['loss'], label='Training Loss')
plt.plot(history_improved_cnn.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history_improved_cnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_improved_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Lưu mô hình
model_improved_cnn.save('/kaggle/working/classification_CNN_model_v2.h5')

# 3. VGG

**Tiền xử lý dữ liệu**

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

# Directories for train, valid, test data
train_data_dir = '/kaggle/input/learntools/train/'
validation_data_dir = '/kaggle/input/learntools/valid/'
test_data_dir = '/kaggle/input/learntools/test/'

# Read data from train, valid, and test sets
train_filenames, train_labels = read_data(os.path.join(train_data_dir, "_annotations.csv"))
test_filenames, test_labels = read_data(os.path.join(test_data_dir, "_annotations.csv"))
validation_filenames, validation_labels = read_data(os.path.join(validation_data_dir, "_annotations.csv"))


# Xử lý dữ liệu hình ảnh từ tập train
train_images = []
train_labels_list = []
for filename, label in zip(train_filenames, train_labels):
    image = cv2.imread(os.path.join(train_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    train_images.append(image)
    train_labels_list.append(label)

train_images = np.array(train_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
train_labels = pd.get_dummies(train_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập valid
valid_images = []
valid_labels_list = []
for filename, label in zip(validation_filenames, validation_labels):
    image = cv2.imread(os.path.join(validation_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    valid_images.append(image)
    valid_labels_list.append(label)

valid_images = np.array(valid_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
valid_labels = pd.get_dummies(valid_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
test_labels = pd.get_dummies(test_labels_list)  # Chuyển đổi nhãn sang one-hot encoding


**Xây dựng mô hình VGG16**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import pandas as pd
import numpy as np
import cv2
import os

def VGG16(input_shape, num_classes):
    model = Sequential()

    # Khối VGG 1
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, name='conv2d'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_1'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d'))

    # Khối VGG 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_2'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_3'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_1'))

    # Khối VGG 3
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_4'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_5'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_6'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_2'))

    # Khối VGG 4
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_7'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_8'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_9'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_3'))

    # Khối VGG 5
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_10'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_11'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_12'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_4'))

    model.add(Flatten(name='flatten'))

    # Fully connected layer
    model.add(Dense(4096, activation='relu', name='dense'))
    model.add(Dropout(0.5, name='dropout'))

    model.add(Dense(4096, activation='relu', name='dense_1'))
    model.add(Dropout(0.5, name='dropout_1'))

    model.add(Dense(num_classes, activation='softmax', name='dense_2'))

    return model

# Dữ liệu huấn luyện, validation, và kiểm thử đã được tiền xử lý và chuẩn bị trước

# Kích thước của ảnh đầu vào
input_shape = (224, 224, 3)

# Số lượng lớp đầu ra (số lượng lớp trong tập dữ liệu)
num_classes = 5

# Tạo mô hình VGG16
model_vgg16 = VGG16(input_shape, num_classes)

# Biên dịch mô hình
model_vgg16.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Hiển thị tổng quan của mô hình
model_vgg16.summary()


In [ ]:
# Huấn luyện mô hình
history_vgg16 = model_vgg16.fit(train_images, train_labels, epochs=20, batch_size=32, validation_data=(valid_images, valid_labels))

In [ ]:
# Biểu đồ loss và accuracy
import matplotlib.pyplot as plt

plt.plot(history_vgg16.history['loss'], label='Training Loss')
plt.plot(history_vgg16.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history_vgg16.history['accuracy'], label='Training Accuracy')
plt.plot(history_vgg16.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Đánh giá mô hình trên tập test
test_loss_vgg16, test_accuracy_vgg16 = model_vgg16.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_vgg16)
print("Test Accuracy:", test_accuracy_vgg16)

In [ ]:
# Lưu mô hình
model_vgg16.save('/kaggle/working/classification_VGG16_model.h5')

# LeNet

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

# Directories for train, valid, test data
train_data_dir = '/kaggle/input/learntools/train/'
validation_data_dir = '/kaggle/input/learntools/valid/'
test_data_dir = '/kaggle/input/learntools/test/'

# Read data from train, valid, and test sets
train_filenames, train_labels = read_data(os.path.join(train_data_dir, "_annotations.csv"))
test_filenames, test_labels = read_data(os.path.join(test_data_dir, "_annotations.csv"))
validation_filenames, validation_labels = read_data(os.path.join(validation_data_dir, "_annotations.csv"))


# Xử lý dữ liệu hình ảnh từ tập train
train_images = []
train_labels_list = []
for filename, label in zip(train_filenames, train_labels):
    image = cv2.imread(os.path.join(train_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    train_images.append(image)
    train_labels_list.append(label)

train_images = np.array(train_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
train_labels = pd.get_dummies(train_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập valid
valid_images = []
valid_labels_list = []
for filename, label in zip(validation_filenames, validation_labels):
    image = cv2.imread(os.path.join(validation_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    valid_images.append(image)
    valid_labels_list.append(label)

valid_images = np.array(valid_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
valid_labels = pd.get_dummies(valid_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
test_labels = pd.get_dummies(test_labels_list)  # Chuyển đổi nhãn sang one-hot encoding


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

def LeNet5(input_shape=(224, 224, 3), num_classes=5):
    model = Sequential()

    # C1 Convolutional layer
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape, padding='same', name='C1'))

    # S2 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))

    # C3 Convolutional layer
    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh', padding='valid', name='C3'))

    # S4 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))

    # C5 Convolutional layer
    model.add(Conv2D(120, kernel_size=(5, 5), activation='tanh', padding='valid', name='C5'))

    model.add(Flatten(name='Flatten'))

    # FC6 Fully connected
    model.add(Dense(84, activation='tanh', name='FC6'))

    # FC7 output
    model.add(Dense(num_classes, activation='softmax', name='FC7'))
    return model

if __name__ == '__main__':
    model_lenet = LeNet5()
    print(model_lenet.summary())


In [ ]:
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Huấn luyện mô hình
def train_model(model, train_images, train_labels, valid_images, valid_labels, batch_size=32, epochs=10):
    # Compile model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define callbacks
    checkpoint = ModelCheckpoint('lenet_model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs,
                        validation_data=(valid_images, valid_labels), callbacks=[checkpoint, early_stopping])

    return history

# Hiển thị biểu đồ đánh giá
def plot_evaluation(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Load preprocessed data
# Replace train_images, train_labels, valid_images, valid_labels with your preprocessed data

# Train the model
history = train_model(model_lenet, train_images, train_labels, valid_images, valid_labels, batch_size=32, epochs=10)

# Plot evaluation
plot_evaluation(history)

# Evaluate on test set
test_loss_lenet, test_accuracy_lenet = model_lenet.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_lenet)
print("Test Accuracy:", test_accuracy_lenet)

In [ ]:
# Lưu mô hình
model_lenet.save('/kaggle/working/lenet_model.keras')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization

def Improved_LeNet():
    model = Sequential()

    # C1 Convolutional layer
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(224, 224, 3), padding='same', name='C1'))
    model.add(BatchNormalization())
    # S2 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))

    # C3 Convolutional layer
    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='valid', name='C3'))
    model.add(BatchNormalization())
    # S4 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))

    # Flatten the feature maps
    model.add(Flatten(name='Flatten'))

    # FC6 Fully connected
    model.add(Dense(120, activation='relu', name='FC6'))
    model.add(BatchNormalization())

    # FC7 Fully connected
    model.add(Dense(84, activation='relu', name='FC7'))
    model.add(BatchNormalization())

    # FC8 output
    model.add(Dense(5, activation='softmax', name='FC8'))

    return model

# Create the improved LeNet model
improved_lenet_model = Improved_LeNet()

# Compile the model
improved_lenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = train_model(improved_lenet_model, train_images, train_labels, valid_images, valid_labels, batch_size=32, epochs=200)

In [ ]:
# Plot evaluation
plot_evaluation(history)

# Evaluate on test set
test_loss_improved_lenet, test_accuracy_improved_lenet = improved_lenet_model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_improved_lenet)
print("Test Accuracy:", test_accuracy_improved_lenet)

# Save the model
improved_lenet_model.save('/kaggle/working/lenet_model_v2.keras')


 # ResNet18

In [ ]:
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def identity_block(X, filters):
    F1, F2, F3 = filters

    # Save the input value (shortcut path)
    shortcut = X

    # Main path
    X = Conv2D(filters=F1, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut value to main path
    X = Add()([X, shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, filters, s=2):
    F1, F2, F3 = filters

    # Save the input value (shortcut path)
    shortcut = X

    # Convolutional shortcut path
    shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(shortcut)
    shortcut = BatchNormalization(axis=3)(shortcut)

    # Main path
    X = Conv2D(filters=F1, kernel_size=(3, 3), strides=(s, s), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut value to main path
    X = Add()([X, shortcut])
    X = Activation('relu')(X)

    return X

def ResNet18(input_shape=(224, 224, 3), num_classes=5):
    input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters=[64, 64, 64], s=1)
    X = identity_block(X, filters=[64, 64, 64])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 128], s=2)
    X = identity_block(X, filters=[128, 128, 128])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 256], s=2)
    X = identity_block(X, filters=[256, 256, 256])

    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 512], s=2)
    X = identity_block(X, filters=[512, 512, 512])

    # Average pooling
    X = GlobalAveragePooling2D()(X)

    # Output layer
    X = Dense(num_classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=input, outputs=X)
    return model

if __name__ == '__main__':
    model_resnet18 = ResNet18()
    print(model_resnet18.summary())


In [ ]:
model_resnet18.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_resnet18.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(valid_images, valid_labels))


In [ ]:
import matplotlib.pyplot as plt

# Biểu đồ độ chính xác
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Biểu đồ mất mát
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss_resnet18, test_accuracy_resnet18  = model_resnet18.evaluate(test_images, test_labels)
print('Test Loss:', test_loss_resnet18)
print('Test Accuracy:', test_accuracy_resnet18)

In [ ]:
# Lưu mô hình vào một tệp tin
model_resnet18.save('/kaggle/working/resnet18_model.h5')

In [ ]:
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def identity_block(X, filters):
    F1, F2, F3 = filters

    # Save the input value (shortcut path)
    shortcut = X

    # Main path
    X = Conv2D(filters=F1, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut value to main path
    X = Add()([X, shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, filters, s=2):
    F1, F2, F3 = filters

    # Save the input value (shortcut path)
    shortcut = X

    # Convolutional shortcut path
    shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(shortcut)
    shortcut = BatchNormalization(axis=3)(shortcut)

    # Main path
    X = Conv2D(filters=F1, kernel_size=(3, 3), strides=(s, s), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut value to main path
    X = Add()([X, shortcut])
    X = Activation('relu')(X)

    return X

def ResNet18_deep(input_shape=(224, 224, 3), num_classes=5):
    input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters=[64, 64, 64], s=1)
    X = identity_block(X, filters=[64, 64, 64])
    X = identity_block(X, filters=[64, 64, 64])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 128], s=2)
    X = identity_block(X, filters=[128, 128, 128])
    X = identity_block(X, filters=[128, 128, 128])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 256], s=2)
    X = identity_block(X, filters=[256, 256, 256])
    X = identity_block(X, filters=[256, 256, 256])
    
    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 512], s=2)
    X = identity_block(X, filters=[512, 512, 512])
    X = identity_block(X, filters=[512, 512, 512])

    # Average pooling
    X = GlobalAveragePooling2D()(X)

    # Output layer
    X = Dense(num_classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=input, outputs=X)
    return model

if __name__ == '__main__':
    model_resnet18_deep = ResNet18_deep()
    print(model_resnet18_deep.summary())


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Compile model
model_resnet18_deep.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = train_model(model_resnet18_deep, train_images, train_labels, valid_images, valid_labels, batch_size=32, epochs=200)

In [ ]:

# Evaluate the model on test set
test_loss_resnet18_deep, test_accuracy_resnet18_deep = model_resnet18_deep.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_resnet18_deep)
print("Test Accuracy:", test_accuracy_resnet18_deep)

# Plot evaluation metrics
plot_evaluation(history)


In [ ]:
# Save the model (version 2)
model_resnet18_deep.save("resnet18_deep_model_v2.h5")

In [ ]:
import pandas as pd

# Tạo DataFrame từ dictionary
test_metrics = {
    "Model": ["MLP", "Improved MLP", "Simple CNN", "Improved CNN", "VGG16", "LeNet", "Improved LeNet", "ResNet18", "ResNet18 Deep"],
    "Test Loss": [test_loss_mlp, test_loss_improved_mlp, test_loss_simple_cnn, test_loss_improved_cnn, test_loss_vgg16, test_loss_lenet, test_loss_improved_lenet, test_loss_resnet18, test_loss_resnet18_deep],
    "Test Accuracy (%)": [test_accuracy_mlp * 100, test_accuracy_improved_mlp * 100, test_accuracy_simple_cnn * 100, test_accuracy_improved_cnn * 100, test_accuracy_vgg16 * 100, test_accuracy_lenet * 100, test_accuracy_improved_lenet * 100, test_accuracy_resnet18 * 100, test_accuracy_resnet18_deep * 100]
}

df_test_results = pd.DataFrame(test_metrics)
print(df_test_results)
# Lưu DataFrame vào file CSV
df_test_results.to_csv('/kaggle/working/model_evaluation.csv', index=False)